# Element volume locking

Relationships between $(\lambda,\mu)$ and $(E, \nu)$ are
\begin{equation}
\lambda=\frac{E\nu}{(1+\nu)(1-2\nu)},\,\,\,\,\mu=\frac{E}{2(1+\nu)}
\end{equation}

Basic formulation introduced above is based on stiffness, but for $\nu\rightarrow 0.5$ Young modulus $E\rightarrow\infty$ and calculations collapse with reaching $\nu=0.5$. This singularity introduces the so called volume locking. It can be seen from simple numerical experiment. Suppose rectangle domain holded on bottom and loaded on top edge and suppose almost critical value of Poisson ratio $\nu=0.4999$. The task is implemented as in *2D_Elasticity* example. The domain response should be symmetrical, but result is ...

In [2]:
# Modified 2D_Elasticity example for nu->infty

To recover the correct physical behavior,
we switch to mixed variational problem formulation.
In the previous example we supposed the stress tensor is
\begin{equation}
    \boldsymbol{\sigma}=\lambda\mathrm{div}(\mathbf{u})\mathbf{I} + 2\mu\boldsymbol{\varepsilon}(\mathbf{u})
\end{equation}
and $\mathbf{u}$ was the unknown field.
Here we define $p=-\lambda\mathrm{div}\mathbf{u}$, the hydrostatic pressure, as an additional unknown.
Hence the previous strong form
\begin{align*}
	\text{div} \boldsymbol{\sigma} + \mathbf{b} &= 0 && \text{in } \Omega, \\
	\boldsymbol{\sigma} \cdot \mathbf{n} &= \mathbf{g} && \text{on } \Gamma_{\text{N}}, \\
	\mathbf{u} &= \bar{\mathbf{u}} && \text{on } \Gamma_{\text{D}},
\end{align*}
is replaced by more equations
\begin{align*}
	- \nabla p + \mathrm{div}(2 \mu \boldsymbol{\varepsilon}(\mathbf{u})) + \mathbf{b} &= 0 && \text{in } \Omega, \\
	\mathrm{div}\mathbf{u} + \frac{p}{\lambda} &= 0 && \text{in } \Omega, \\
	p \mathbf{n} + 2 \mu \boldsymbol{\varepsilon}(\mathbf{u}) \cdot \mathbf{n} &= \mathbf{g} && \text{on } \Gamma_{\text{N}}, \\
	\mathbf{u} &= \bar{\mathbf{u}} && \text{on } \Gamma_{\text{D}},
\end{align*}
Its weak formulation being: find $(\mathbf{u},p)$ s.t.
\begin{align*}
    -\int_\Omega p \text{ div}\, \mathbf{v} \,\mathrm{d}X
    +
    \int_\Omega 2 \mu \boldsymbol{\varepsilon}(\mathbf{u}) : \boldsymbol{\varepsilon}(\mathbf{v}) \,\mathrm{d}X
    &= \int_\Omega \mathbf{b} \cdot \mathbf{v} \,\mathrm{d}X
    + \int_{\partial \Omega} \mathbf{g} \cdot \mathbf{v} \,\mathrm{d}S,
    \quad \forall \mathbf{v}, \\
    \int_\Omega \left(\mathrm{div}\mathbf{u}+\frac{p}{\lambda} \right) q \,\mathrm{d}X
    &=
    0,
    \quad \forall q.
\end{align*}
It is an example of Hellinger-Reissner mixed formulation
and the existence is assured e.g. by the Babuška-Brezi condition.

### Implementation

Since FEniCS requires only one form, we rewrite the system above to a one variational equality
\begin{equation}
    -\int_\Omega p \text{ div}\, \mathbf{v} \,\mathrm{d}X
    +
    \int_\Omega 2 \mu \boldsymbol{\varepsilon}(\mathbf{u}) : \boldsymbol{\varepsilon}(\mathbf{v}) \,\mathrm{d}X
    - \int_\Omega \left(\mathrm{div}\mathbf{u}+\frac{p}{\lambda} \right) q \,\mathrm{d}X
    = \int_\Omega \mathbf{b} \cdot \mathbf{v} \,\mathrm{d}X
    + \int_{\partial \Omega} \mathbf{g} \cdot \mathbf{v} \,\mathrm{d}S,
    \,\forall \mathbf{v},\,\forall q.
\end{equation}
We have to also choose the discrete subspaces carefully in order to keep the Babuška-Brezi condition valid,
even uniformly with respect to the discretization parameter;
see the list of suitable pairs of function spaces in [Hughes, T. J. R. - The Finite Element Method (1987), Chapter 4, pp. 201]

First of all it is necessary redefine stress function using new variable $p$.

Definitions of material constants remains the same. It holds also for mesh.

The crucial aspect of mixed formulation id definition of Function space. The space for displacements is vector-valued meanwhile function space for pressure variable is scalar. In addition, it is appropriate to select different degrees of space for each variable to maintain Babuška–Brezzi inf-sup condition. To implement such space in FEniCS it is necessary define individual element objects:

where first stands for second-order polynomial vector-valued element and second stands for first-order polynomial scalar element. Next step is definition of mixed element by calling *MixedElement* constructor with array of individual elements.

And finally mixed function space is created by *FunctionSpace* object with mixed element as argument.

Dirichlet boundary condition are defined by classic way. In this case the space *V* repsesents whole mixed function space, instead of *V.sub(0)* represents vector-valued displacement subspace and *V.sub(1)* represents scalar space for pressure.

It is also possible to access to individual subspaces of subspace. For example following *V.sub(0).sub(1)* is space for $y$-displacements.

Calling *fe.TestFunction(V)* in mixed formulation returns test function on whole mixed space. It is possible to extract test functions from individual spaces by function *fe.split()*, but simpler solution is calling *fe.TestFunctions(V)* instead of *fe.TestFunction(V)*. It returns splitted functions of individual spaces. The same holds for trial functions.

The weak form presented below is for nearly incompressible continuum. For full incompressibility, it is neccessary to delete $p/\lambda$ member.

Solution is obtained by classic way.

To postprocess solution, it is possible access to subspaces by *.sub(n)*.

### Subspace

In this example solution is saved in *sol* variable as mixed function. To acces individual subfunctions, FEniCS provides function *split*. There exists two different version of this function. The first is called via FEniCS package function as

It returns some kind of link on subspace. It is usually sufficient for plotting for example, but may not be sufficient for some operations with subfunctions. Therefore FEniCS provides another split function.

It returns two subfunctions of type *Function*. 

### Code